In [44]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from scipy.sparse import hstack
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\migue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\migue\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
df = pd.read_csv(r'../Data/data_modelo.csv')
df

,title,tokenizada_overview,tokenizada_title,tokenizada_director_name,tokenizada_production_companies_names
0,Toy Story,led woody andy toys live happily room andy bir...,toy story,john lasseter,animation studios
1,Jumanji,siblings judy peter discover enchanted board g...,jumanji,joe johnston,pictures film communications
2,Grumpier Old Men,family wedding reignites ancient feud neighbor...,grumpier old men,howard deutch,gate
3,Waiting to Exhale,cheated mistreated stepped women holding breat...,waiting exhale,forest whitaker,century fox film corporation
4,Father of the Bride Part II,george banks recovered daughter wedding receiv...,father bride part ii,charles shyer,productions pictures
...,...,...,...,...,...
43267,Robin Hood,yet another version classic epic enough variat...,robin hood,john irvin,rundfunk wdr title films century fox televisio...
43268,Century of Birthing,artist struggles finish work storyline cult pl...,century birthing,lav diaz,olivia
43269,Betrayal,one hits goes wrong professional assassin ends...,betrayal,mark lester,world pictures
43270,Satan Triumphant,small town live two brothers one minister one ...,satan triumphant,yakov protazanov,NaN


In [36]:
df

,title,tokenizada_overview,tokenizada_title,tokenizada_director_name,tokenizada_production_companies_names
0,Toy Story,led woody andy toys live happily room andy bir...,toy story,john lasseter,animation studios
1,Jumanji,siblings judy peter discover enchanted board g...,jumanji,joe johnston,pictures film communications
2,Grumpier Old Men,family wedding reignites ancient feud neighbor...,grumpier old men,howard deutch,gate
3,Waiting to Exhale,cheated mistreated stepped women holding breat...,waiting exhale,forest whitaker,century fox film corporation
4,Father of the Bride Part II,george banks recovered daughter wedding receiv...,father bride part ii,charles shyer,productions pictures
...,...,...,...,...,...
43267,Robin Hood,yet another version classic epic enough variat...,robin hood,john irvin,rundfunk wdr title films century fox televisio...
43268,Century of Birthing,artist struggles finish work storyline cult pl...,century birthing,lav diaz,olivia
43269,Betrayal,one hits goes wrong professional assassin ends...,betrayal,mark lester,world pictures
43270,Satan Triumphant,small town live two brothers one minister one ...,satan triumphant,yakov protazanov,NaN


In [37]:
df = df.fillna("")

In [38]:
lista_columnas = ['tokenizada_overview', 'tokenizada_title', 'tokenizada_director_name', 'tokenizada_production_companies_names']

lista_matrices= []
dic_vectores = {}

for column in lista_columnas:
    vectorizer = TfidfVectorizer()
    matriz = vectorizer.fit_transform(df[column])
    dic_vectores[column]=vectorizer
    lista_matrices.append(matriz)



In [39]:
combinacion_matrices = hstack(lista_matrices).tocsr() if len(lista_matrices) > 1 else lista_matrices[0]


In [40]:
def similitud_coseno(idx, matriz):
    return cosine_similarity(matriz[idx], matriz).flatten()

In [41]:
def recomendador(title, data, num=5):
    if title not in df['title'].values:
        return f'La película {title} aun no se ha realizado o usted no sabe escribir'

    idx = data[data['title']==title].index[0]
    puntaje = similitud_coseno(idx, combinacion_matrices)

    lista_pelis = list(enumerate(puntaje))
    lista_pelis = sorted(lista_pelis, key=lambda x: x[1], reverse=True)

    lista_pelis= lista_pelis[1:num+1]

    indices = [i[0] for i in lista_pelis]

    lista_vacia = []

    return data['title'].iloc[indices].tolist()

In [43]:
recomendador('Scary Movie', df)

['Scary Movie 2',
 'Scary Movie 3',
 'Scary Movie 4',
 'Scary Movie 5',
 'A Low Down Dirty Shame']